In [1]:
pip install --upgrade pyod  

In [2]:
pip install combo

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np 
import pandas as pd 

import os

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor

from scipy.stats.mstats import winsorize

In [5]:
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.lof import LOF

In [6]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install lightgbm 

Note: you may need to restart the kernel to use updated packages.


In [9]:
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import catboost
import lightgbm

In [10]:
#CSV to Load
FMR_19_data= "FY19_FMR.csv"
FMR19_df=pd.read_csv(FMR_19_data, na_values=['999.99'])
FMR19_df.head()
FMR19_df.tail()

,fips2010,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4,state,metro_code,areaname,county,cousub,countyname,county_town_name,pop2010,acs_2018_2,state_alpha,fmr_type,metro,fmr_pct_chg,fmr_dollar_chg
4762,7215199999,431,459,549,728,913,72,METRO41980MM7440,"San Juan-Guaynabo, PR HUD Metro FMR Area",151,99999,Yabucoa Municipio,Yabucoa Municipio,37941,530,PR,50,1,0.035849,19
4763,7215399999,354,363,416,575,731,72,METRO38660M49500,"Yauco, PR HUD Metro FMR Area",153,99999,Yauco Municipio,Yauco Municipio,42043,389,PR,50,1,0.069409,27
4764,7801099999,779,794,970,1215,1311,78,NCNTY78010N78010,"St. Croix Island, VI",10,99999,St. Croix,St. Croix,53234,916,VI,50,0,0.058952,54
4765,7802099999,1034,1225,1523,1908,2059,78,NCNTY78020N78020,"St. John Island, VI",20,99999,St. John,St. John,4197,1439,VI,50,0,0.058374,84
4766,7803099999,744,897,1123,1407,1518,78,NCNTY78030N78030,"St. Thomas Island, VI",30,99999,St. Thomas,St. Thomas,51181,1061,VI,50,0,0.058435,62


In [11]:
FMR19_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4767 entries, 0 to 4766
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fips2010          4767 non-null   int64  
 1   fmr_0             4767 non-null   int64  
 2   fmr_1             4767 non-null   int64  
 3   fmr_2             4767 non-null   int64  
 4   fmr_3             4767 non-null   int64  
 5   fmr_4             4767 non-null   int64  
 6   state             4767 non-null   int64  
 7   metro_code        4767 non-null   object 
 8   areaname          4767 non-null   object 
 9   county            4767 non-null   int64  
 10  cousub            4767 non-null   int64  
 11  countyname        4767 non-null   object 
 12  county_town_name  4767 non-null   object 
 13  pop2010           4767 non-null   int64  
 14  acs_2018_2        4767 non-null   int64  
 15  state_alpha       4767 non-null   object 
 16  fmr_type          4767 non-null   int64  


In [12]:
FMR19_df.isnull().sum()

fips2010            0
fmr_0               0
fmr_1               0
fmr_2               0
fmr_3               0
fmr_4               0
state               0
metro_code          0
areaname            0
county              0
cousub              0
countyname          0
county_town_name    0
pop2010             0
acs_2018_2          0
state_alpha         0
fmr_type            0
metro               0
fmr_pct_chg         0
fmr_dollar_chg      0
dtype: int64

In [13]:
print("Categorical Columns:", list(FMR19_df.select_dtypes(include = 'object').columns))
print("Numerical Columns:", list(FMR19_df.select_dtypes(exclude = 'object').columns))

Categorical Columns: ['metro_code', 'areaname', 'countyname', 'county_town_name', 'state_alpha']
Numerical Columns: ['fips2010', 'fmr_0', 'fmr_1', 'fmr_2', 'fmr_3', 'fmr_4', 'state', 'county', 'cousub', 'pop2010', 'acs_2018_2', 'fmr_type', 'metro', 'fmr_pct_chg', 'fmr_dollar_chg']


In [14]:
#

In [15]:
FMR19_df[FMR19_df['fmr_0']==0]['fmr_0'].count()

0

In [16]:
#FMR21_studio_df=FMR21_df[FMR21_df['fmr_0']!=0]
#FMR21_studio_df.head()

In [17]:
CFMR19_df = FMR19_df.drop(['state','metro_code', 'fips2010','areaname','county_town_name','metro', 'cousub','county','fmr_type','fmr_pct_chg','fmr_dollar_chg','acs_2018_2'], axis=1)
FMR19_df.head()

,fips2010,fmr_0,fmr_1,fmr_2,fmr_3,fmr_4,state,metro_code,areaname,county,cousub,countyname,county_town_name,pop2010,acs_2018_2,state_alpha,fmr_type,metro,fmr_pct_chg,fmr_dollar_chg
0,100199999,581,696,825,1043,1436,1,METRO33860M33860,"Montgomery, AL MSA",1,99999,Autauga County,Autauga County,54571,829,AL,50,1,-0.004825,-4
1,100399999,756,773,888,1252,1501,1,METRO19300M19300,"Daphne-Fairhope-Foley, AL MSA",3,99999,Baldwin County,Baldwin County,182265,879,AL,50,1,0.010239,9
2,100599999,500,503,666,838,941,1,NCNTY01005N01005,"Barbour County, AL",5,99999,Barbour County,Barbour County,27457,657,AL,50,0,0.013699,9
3,100799999,706,763,873,1154,1282,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",7,99999,Bibb County,Bibb County,22915,882,AL,50,1,-0.010204,-9
4,100999999,706,763,873,1154,1282,1,METRO13820M13820,"Birmingham-Hoover, AL HUD Metro FMR Area",9,99999,Blount County,Blount County,57322,882,AL,50,1,-0.010204,-9


In [18]:
CFMR19_df.rename(columns = {'fmr_0':'Studio', 'fmr_1':'One Bedroom', 'fmr_2':'Two Bedroom', 'fmr_3':'Three Bedroom','fmr_4':'Four Bedroom','pop2010':'Population','countyname':'County','state_alpha':'State'}, inplace = True)

In [19]:
CFMR19_df=CFMR19_df.groupby(by=['State'], dropna=False).mean()

In [20]:
CFMR19_df.insert(0, 'Year', '2019')

In [21]:
CFMR19_df=CFMR19_df.round(2)

In [23]:
CFMR19_df.head()
CFMR19_df.to_csv(r"CFMR19_df.csv")

In [ ]:
numeric_columns = list(FMR19_df.select_dtypes(exclude = 'object').columns)
fig = make_subplots(rows=2, cols=3, shared_yaxes=True)
index = 0
for row in range(1,3):
    for col in range(1,4):
        fig.add_trace(go.Box(x=FMR19_df[numeric_columns[index]], name = numeric_columns[index]),
                      row = row,
                      col = col,
)
        index = index + 1
fig.update_layout(height=600, width=1300,
                  title_text="Box-Plots of Numerical Variables")

fig.show()

In [ ]:
sns.regplot(x="Studio", y="Population", data=FMR19_df)
plt.ylim(0,)